In [1]:
#起動コード
import boto3
from boto3.session import Session

session = Session(profile_name='tokyo',region_name='ap-northeast-1')
ec2 = boto3.resource('ec2')

instance_id = 'i-05cc604e17f8cec3f'
instance = ec2.Instance(instance_id)
response = instance.start()
print(f"{instance_id}を起動しました。")

i-05cc604e17f8cec3fを起動しました。


In [2]:
#停止コード
import boto3
from boto3.session import Session

session = Session(profile_name='tokyo',region_name='ap-northeast-1')
ec2 = boto3.resource('ec2')

instance_id = 'i-05cc604e17f8cec3f'
instance = ec2.Instance(instance_id)
response = instance.stop()
print(f"{instance_id}を停止しました。")


i-05cc604e17f8cec3fを停止しました。


In [3]:
#EC2の状況を見て起動・停止・状態確認をするコマンド

import boto3
from boto3.session import Session

session = Session(profile_name='tokyo',region_name='ap-northeast-1')

ec2 = boto3.resource('ec2')
instance_id = 'i-05cc604e17f8cec3f'

instance = ec2.Instance(instance_id)
current_state = instance.state['Name']
print(f"現在の状況： {current_state}")

if current_state == 'stopped':
    instance.start()
    print(f"{instance_id}を起動しました。")
elif current_state == 'running':
    instance.stop()
    print(f"{instance_id}を停止しました。")
else:
    print(f"{instance_id}は現在{current_state}状態のため、操作をスキップしました。")


現在の状況： stopping
i-05cc604e17f8cec3fは現在stopping状態のため、操作をスキップしました。


In [4]:
#複数インスタンスの状態確認・それに対する起動・停止コード

import boto3
from boto3.session import Session

session = Session(profile_name='tokyo',region_name='ap-northeast-1')
instance_ids = ['i-05cc604e17f8cec3f','i-0e6553baefebb744b']

ec2 = boto3.resource('ec2')
for instance_id in instance_ids:
    instance = ec2.Instance(instance_id)
    state = instance.state['Name']
    print(f"{instance_id}の現在の状態：{state}")

    if state =='stopped':
        instance.start()
        print(f"{instance_id}を起動しました。")
    elif state == 'running':
        instance.stop()
        print(f"{instance_id}を停止しました。")
    else:
        print(f"{instance_id}は現在{state}状態のため、操作をスキップしました。")
            

i-05cc604e17f8cec3fの現在の状態：stopping
i-05cc604e17f8cec3fは現在stopping状態のため、操作をスキップしました。
i-0e6553baefebb744bの現在の状態：stopped
i-0e6553baefebb744bを起動しました。


In [6]:
#コード・タグ指定でフィルタしてEC2起動・停止

import boto3
from boto3.session import Session

session = Session(profile_name='tokyo',region_name='ap-northeast-1')
target_names = ['lms','migration-ojt-honnbann1']
ec2 = boto3.resource('ec2')

instances = ec2.instances.all()

for instance in instances:
    tags ={tag['Key']: tag['Value'] for tag in instance.tags or []}
    name = tags.get('Name','')
    print(f"チェック中：{instance.id} タグ名：{name}")
    
    if name in target_names:
        state = instance.state['Name']
        print(f"{name}({instance.id})の現在の状態：{state}")

        if state == 'stopped':
            instance.start()
            print(f"{name}({instance.id})を起動しました。")
        elif state == 'running':
            instance.stop()
            print(f"{name}({instance.id})を停止しました。")
        else:
            print(f"{name}({instance.id})は現在{state}状態のため、操作をスキップしました。")    


チェック中：i-05cc604e17f8cec3f タグ名：migration-ojt-honnbann1
migration-ojt-honnbann1(i-05cc604e17f8cec3f)の現在の状態：stopping
migration-ojt-honnbann1(i-05cc604e17f8cec3f)は現在stopping状態のため、操作をスキップしました。
チェック中：i-0e6553baefebb744b タグ名：lms
lms(i-0e6553baefebb744b)の現在の状態：stopping
lms(i-0e6553baefebb744b)は現在stopping状態のため、操作をスキップしました。
チェック中：i-035cd2d49fa0614bd タグ名：linux-test-20250411


In [ ]:
#EC2の最新状態一覧をcsvに出力

import boto3
import csv
from boto3.session import Session

session = Session(profile_name='tokyo',region_name='ap-northeast-1')

ec2 = boto3.resource('ec2')

csv_file = 'ec2_status_report.csv'

data = [['Instance ID','Name','State']]

instances = ec2.instances.all()
for instance in instances:
    instance_id = instance.id
    state = instance.state['Name']

    tags = {tag['Key']:tag['Value'] for tag in instance.tags or []}
    name = tags.get('Name','')

    data.append([instance_id,name,state])

with open(csv_file,mode='w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(data)

print(f"インスタンス情報を{csv_file}に出力しました。")

インスタンス情報をec2^auto/ec2_status_report.csvに出力しました。


In [ ]:
#状態がstoppedのものだけ出力

import boto3
import csv
from boto3.session import Session

session = Session(profile_name='tokyo',region_name='ap-northeast-1')

ec2 = boto3.resource('ec2')

csv_file = 'stopped_ec2_report.csv'
data = [['Instance_id','Name','State']]

instances = ec2.instances.all()

for instance in instances:
    instance_id = instance.id
    state = instance.state['Name']

    if state =='stopped':
        tags = {tag['Key']:tag['Value'] for tag in instance.tags or []}
        name = tags.get('Name','')

        data.append([instance_id,name,state])

with open(csv_file,mode='w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(data)
print(f"停止中のインスタンスを{csv_file}に出力しました。")

停止中のインスタンスをstopped_ec2_report.csvに出力しました。


In [ ]:
#EC2の状態をCSVに出力＆タグチェック＆台数集計

import boto3
import csv
from datetime import datetime
from boto3.session import Session

session = Session(profile_name='tokyo',region_name='ap-northeast-1')

ec2 = boto3.resource('ec2')
instances = ec2.instances.all()

today = datetime.now().strftime('%Y-%m-%d')
csv_file = f'ec2_summary_report_{today}.csv'

data = [['Report Date',today],[]]
data.append(['Instance ID','Name','State'])

running_count = 0
stopped_count = 0

for instance in instances:
    instance_id = instance.id
    state = instance.state['Name']

    if state == 'running':
        running_count += 1
    elif state == 'stopped':
        stopped_count += 1
    
    tags = {tag['Key']:tag['Value'] for tag in instance.tags or []}
    name = tags.get('Name','(なし)')

    data.append([instance_id,name,state])

data.append([])
data.append(['Summary'])
data.append(['Running Instance',running_count])
data.append(['Stopped Instance',stopped_count])

with open(csv_file, mode='w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(data)

print(f"レポートを{csv_file}に出力しました。")


FileNotFoundError: [Errno 2] No such file or directory: 'ec2-auto/ec2_summary_report_2025-04-24.csv'